`torch.cuda.get_device_capability()` 함수를 사용하여 현재 CUDA 장치의 major 버전과 minor 버전을 조회합니다.


In [6]:
import torch

# CUDA major, minor 버전 확인
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 0)

In [7]:
# unsloth 설치
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7bus3z00/unsloth_7ec814d8210044818ea1f9677b52f814
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7bus3z00/unsloth_7ec814d8210044818ea1f9677b52f814
  Resolved https://github.com/unslothai/unsloth.git to commit b9067f98d124173b354808b116f931410e5b699c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


`FastLanguageModel.from_pretrained` 함수를 사용하여 사전 훈련된 언어 모델을 로드하는 과정을 설명합니다.

- 최대 시퀀스 길이(`max_seq_length`)를 설정하여 모델이 처리할 수 있는 입력 데이터의 길이를 지정합니다.
- 데이터 타입(`dtype`)은 자동 감지되거나, 특정 하드웨어에 최적화된 형식(`Float16`, `Bfloat16`)으로 설정할 수 있습니다.
- 4비트 양자화(`load_in_4bit`) 옵션을 사용하여 메모리 사용량을 줄일 수 있으며, 이는 선택적입니다.
- 사전 정의된 4비트 양자화 모델 목록(`fourbit_models`)에서 선택하여 다운로드 시간을 단축하고 메모리 부족 문제를 방지할 수 있습니다.
- `FastLanguageModel.from_pretrained` 함수를 통해 모델과 토크나이저를 로드하며, 이때 모델 이름(`model_name`), 최대 시퀀스 길이, 데이터 타입, 4비트 로딩 여부를 매개변수로 전달합니다.
- 선택적으로, 특정 게이트 모델을 사용할 경우 토큰(`token`)을 제공할 수 있습니다.


In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화 사용
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  #https://huggingface.co/unsloth 에서 확인

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # 모델명 설정
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    dtype=dtype,  # 데이터 타입을 설정
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부 설정
)

==((====))==  Unsloth 2025.4.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

beomi/Llama-3-Open-Ko-8B-Instruct-preview does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


전통적인 FFNN은 보통 다음과 같은 형태
FFN(x) = Linear2(Activation(Linear1(x)))

즉, 입력 x를 첫 번째 선형 레이어(Linear1)로 차원을 확장하고,

활성화 함수(예: ReLU)를 적용한 뒤, 두 번째 선형 레이어(Linear2)로 다시 원래 차원으로 축소하는 구조

최근 LLM들, 특히 Llama 계열에서는 SwiGLU (Swish Gated Linear Unit) 또는 이와 유사한 게이트 메커니즘을 사용하는 FFNN 구조

이러한 구조에서 gate_proj, up_proj, down_proj 용어가 사용

Llama 모델의 FFN (SwiGLU FFN)은 대략 다음과 같은 형태로 동작

FFN(x) = down_proj(SiLU(gate_proj(x)) * up_proj(x))

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # rank 설정 8, 16, 32, 64, 128 권장
    lora_alpha=32,  # LoRA 알파 값(Scaling)을 설정
    lora_dropout=0.05,  # 드롭아웃 설정
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    # True 또는 "unsloth"를 사용하여 긴 컨텍스트에 대해 VRAM을 30% 덜 사용, 2배 더 큰 배치 크기 지원
    use_gradient_checkpointing="unsloth",
    random_state=123,  # 난수 상태 설정
    use_rslora=False,  # 순위 안정화 LoRA(lora_alpha / r 대신 lora_alpha / sqrt(r))
    loftq_config=None,  # LoftQ(QLoRA 개선)를 지원
)

### 데이터 준비

In [10]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰. 반드시 추가 필요
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항
    outputs = examples["output"]  # 출력값
    texts = []  # 포맷팅된 텍스트를 저장할 리스트
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 반드시 추가 필요(무한 생성 방지)
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


# Huggingface의 "teddylee777/QA-Dataset-mini" 데이터셋 로드
dataset = load_dataset("teddylee777/QA-Dataset-mini", split="train")


# 데이터셋에 formatting_prompts_func 함수 적용 및 배치 처리
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

### 모델 훈련

Huggingface TRL의 `SFTTrainer`를 사용

- 참고 문서: [TRL SFT 문서](https://huggingface.co/docs/trl/sft_trainer)


In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩 위치 설정

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수
        do_eval=True,
        #evaluation_strategy="steps",
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

In [12]:
# 현재 메모리 상태
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
15.346 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()  # 모델 훈련

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16 | Num Epochs = 50 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bjkim2004 (wooriai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.082800
2,2.800300
3,3.000300
4,2.476800
5,2.214500
6,1.681700
7,1.285100
8,1.205700
9,0.928600
10,0.856800


In [14]:
# 최종 메모리 및 시간 통계
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위
print(
    # 훈련에 사용된 시간을 분 단위
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율

457.2935 seconds used for training.
7.62 minutes used for training.
Peak reserved memory = 15.564 GB.
Peak reserved memory for training = 0.218 GB.
Peak reserved memory % of max memory = 39.346 %.
Peak reserved memory for training % of max memory = 0.551 %.


### 추론



`TextStreamer`를 사용하여 연속적인 추론을 수행

`StoppingCriteria`와 `StoppingCriteriaList`를 사용하여 특정 토큰에서 생성을 중단하는 방법을 구현

- `StopOnToken` 클래스는 `StoppingCriteria`를 상속받아, 생성 중 특정 토큰(`stop_token_id`)이 나타나면 생성을 중단
- `stop_token` 변수에 중단할 토큰을 문자열로 지정
- `tokenizer.encode` 메소드를 사용하여 `stop_token`을 해당 언어 모델의 토큰 ID로 변환
- `StoppingCriteriaList`에 `StopOnToken` 인스턴스를 포함시켜, 생성 과정에서 이를 중단 조건으로 사용


In [15]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정

(예시1)

In [17]:
from transformers import TextStreamer
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "랭체인 튜토리얼 공부할만한 사이트는?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
랭체인 튜토리얼 공부할만한 사이트는?

### Response:
테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314<|end_of_text|>


(예시 2)

In [18]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "피보나치 수열을 이어가세요.(최대 10개)",  # 지시사항
            "1, 1, 2, 3, 5, 8",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
피보나치 수열을 이어가세요.(최대 10개)

### Response:
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233<|end_of_text|>


In [19]:
model.save_pretrained("Llama-3-Open-Ko-8B-finetuned")  # 모델 저장